In [16]:
import torch
from torch import nn
from torch.nn import init

net = nn.Sequential(nn.Linear(4,3), nn.ReLU(),nn.Linear(3,1))

print(net)

X = torch.rand(2,4)
Y = net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


In [2]:
print(type(net.named_parameters()))
for name, param in net.named_parameters():
  print(name, param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [3]:
for name, param in net[0].named_parameters():
  print(name, param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [17]:
class MyModel(nn.Module):
  def __init__(self,**kwargs):
    super(MyModel,self).__init__(**kwargs)
    self.weight1 = nn.Parameter(torch.rand(20,20))
    self.weight2 = torch.rand(20,20)
  
  def fowward(self, x):
    pass

n = MyModel()
for name, param in n.named_parameters():
  print(name)

# params = list(net[0].parameters())
weigh_0 = list(net[0].parameters())[0]
print(weigh_0.data)
print(weigh_0.grad)

Y.backward()
print(weigh_0.grad)

weight1
tensor([[-0.3291,  0.3317,  0.0394,  0.1128],
        [ 0.0705, -0.3498,  0.2080,  0.3114],
        [-0.1096,  0.3058, -0.1165,  0.2111]])
None
tensor([[ 0.0235,  0.0849,  0.0888,  0.0347],
        [-0.1183, -0.0223, -0.1131, -0.1367],
        [ 0.3178,  0.3972,  0.5820,  0.3988]])


In [18]:
for name, param in net.named_parameters():
  if "weight" in name:
    init.normal_(param, mean=0, std=0.01)
    print(name, param.data)


0.weight tensor([[-0.0004,  0.0018,  0.0072,  0.0017],
        [ 0.0027,  0.0107,  0.0090, -0.0050],
        [-0.0185,  0.0055,  0.0091,  0.0154]])
2.weight tensor([[-0.0040, -0.0015,  0.0086]])


In [19]:
for name, param in net.named_parameters():
  if "bias" in name:
    init.constant_(param, val=0)
    print(name, param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


In [20]:
# 自定义初始化方法
# 需要的初始化方法并没有在init模块中提供


def normal_(tensor, mean=0, std=1):
  with torch.no_grad():
    return tensor.normal_(mean, std)

def init_weight_(tensor):
  with torch.no_grad():
    tensor.uniform_(-10,10)
    tensor *= (tensor.abs()>=5).float()

for name, param in net.named_parameters():
  if "weight" in name:
    init_weight_(param)
    print(name, param.data)

0.weight tensor([[-0.0000,  7.0045, -6.5527, -0.0000],
        [ 7.5509,  0.0000, -6.8486, -9.6573],
        [-6.7627,  6.4909, -0.0000,  9.9846]])
2.weight tensor([[-6.4279, -9.0198,  9.7344]])


In [21]:
#共享模型参数
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear,linear)
print(net)

for name, param in net.named_parameters():
  init.constant_(param, val=3)
  print(name, param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [22]:
# 在内存中，这两个线性层起始一个对象
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))

True
True


In [23]:
x = torch.ones(1,1)
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad)

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])
